In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
os.chdir("/home/573/rh2942/WASSA-2023-EMP") # changing dir for evaluation file
# print(os.getcwd())

In [3]:
raw_train = pd.read_csv('./dataset/WASSA23_essay_level_with_labels_train.tsv', sep='\t', na_values='unknown', header=0) # raw csv file consists of 'unknown' values
raw_train.sample(2)

,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,...,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,speaker_number,split,essay_id,emotion
365,465,18,This incident made me feel bad. I assume it w...,4.666667,4.500,48,1.0,6.0,1.0,41.0,...,2.0,1.0,3.286,4.429,2.714,2.571,1,train,464,Sadness
129,182,305,I just read an article about suicides in Russi...,3.833333,2.625,25,1.0,4.0,1.0,29.0,...,6.0,5.5,5.000,2.286,3.286,4.143,1,train,181,Neutral


In [4]:
train_dev = pd.read_csv("./processed_data/essay_article_id_train_dev.csv", index_col=0)
train_dev.sample(2)

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,...,emotion,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
539,207,201,I'm so tired of all this violence. If a cop sh...,44,2.0,6.0,1.0,46.0,22000.0,2,...,Anger,6.5,5.5,2.5,5.0,5.0,3.286,2.286,4.857,4.714
890,20,336,It's so sad to read this kind of story... Poor...,68,2.0,2.0,1.0,21.0,20000.0,2,...,Sadness,5.0,3.0,5.0,4.0,3.5,2.714,3.000,3.143,3.286


In [3]:
raw_dev = pd.read_csv('./dataset/dev/WASSA23_essay_level_dev.tsv', sep='\t', header=0)
raw_dev.sample(2)

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,split,essay_id
17,59,72,Any tyoe of animal poaching and killing just r...,96,2,3,1,27,25000,1,dev,58
112,23,281,I read this awful article today about this col...,89,2,7,1,35,60000,2,dev,522


In [26]:
complete_dev = pd.read_csv('./processed_data/essay_dev_complete.csv', index_col=0, header=0)
complete_dev.sample(2)

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,...,emotion,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
69,310,48,I just read the article about Aleppo and Syria...,65,2,6,1,62,29000,1,...,Sadness,6.5,2.5,1.0,6.5,2.0,3.571,4.0,2.143,4.429
194,425,133,What is human nature coming to? Zoo visitors...,65,2,6,1,62,29000,2,...,Anger/Sadness,6.5,2.5,1.0,6.5,2.0,3.571,4.0,2.143,4.429


In [6]:
raw_test = pd.read_csv('./dataset/test/WASSA23_essay_level_test.tsv', sep='\t', header=0)
raw_test.sample(3)

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,split,essay_id
32,1102,13,This article is a bit silly. I still don't rea...,49,1,6,1,37,12000,2,test,32
42,1110,14,This is devastating news. An airline employee ...,37,1,6,1,36,75000,2,test,42
95,1183,19,Check out this story. It is an outrage that so...,143,2,6,1,42,50000,1,test,95


# Summarise long articles

In [35]:
article_raw = pd.read_csv('./dataset/articles_adobe_AMT.csv', header=0)
article_raw.sample(2)

,article_id,text
243,244,Orlando gunman known to FBI shows difficulty o...
359,360,Trump’s dark “gift” makes victims of us all — ...


In [16]:
import transformers as trf

In [ ]:
checkpoint = "philschmid/flan-t5-base-samsum"
tokeniser = trf.AutoTokenizer.from_pretrained(checkpoint, model_max_length=25000) #25K because article text has maximum ~21K characters

summariser = trf.pipeline(task="summarization", model=checkpoint, tokenizer=tokeniser, device="cuda:0")
 
# Replacing long article by its summary
for index, _ in article_raw.iterrows():
    article_raw.loc[index, "text"] = summariser(article_raw.loc[index, "text"])[0]["summary_text"] # 0-index to take the one and only first paraphrased item

article_raw.to_csv("./processed_data/articles_adobe_AMT_summarised.csv")

# Numerical to textual conversion

In [4]:
# Summarised article
article = pd.read_csv('./processed_data/articles_adobe_AMT_summarised.csv', header=0, index_col=0)
article.sample(2)

,text
article_id,
417,Susan R. Smith lost her best companion this we...
362,Turkish President Recep Tayyip Erdogan sharply...


In [5]:
def num_to_text(raw_data):
    input_data = raw_data.copy() #mandatory step as dataframe is mutable
    
    input_data.dropna(inplace=True)
    print(f"Existing non-numeric columns:\n {input_data.select_dtypes(exclude=['number']).columns.tolist()}") #Just checking if any non-numeric value exist on numeric columns
    
    #converting article id to corresponding article texts
    input_data['article'] = input_data['article_id'].apply(lambda x: article.loc[x, 'text'])
    
    categories = {1.: "I am male.", 2.: "I am female.", 5: "I am nonbinary."} #5 is in WSSA 2022 dataset
    input_data['gender'] = input_data['gender'].map(categories)

    categories = {
        1.: "My education level is Less than a high school diploma.",
        2.: "My education level is High School diploma.",
        3.: "My education level is echnical/Vocational School.",
        4.: "My education level is Some college but no degree.",
        5.: "My education level is Two year associate degree.",
        6.: "My education level is Four year bachelor’s degree.",
        7.: "My education level is Postgradute or professional degree."
    }
    input_data['education'] = input_data['education'].map(categories)

    categories = {
        1.: "My race is White.",
        2.: "My race is Hispanic or Latino.",
        3.: "My race is Black or African American.",
        4.: "My race is Native American or American Indian.",
        5.: "My race is Asian/Pacific Islander.",
        6.: "My race is other."
    }
    input_data['race'] = input_data['race'].map(categories)

    input_data['age'] = input_data['age'].apply(lambda x: "My age is "+str(x)+" years.")
    input_data['income'] = input_data['income'].apply(lambda x: "My income is "+str(x)+".")
    
    input_data['article_id_text'] = input_data['article_id'].apply(lambda x: "I read newspaper article "+str(x)+".")
    
    # print(input_data.isna().any())
    assert input_data.isna().any().any() == False #no NA values
    assert input_data.isnull().any().any() == False #no null values
    
    input_data['demographic'] = input_data['gender'] + ' ' + input_data['age'] + ' ' + input_data['education'] + ' ' + input_data['race'] + ' ' + input_data['income']  + ' ' + input_data['article_id_text']  
    input_data['demographic_essay'] = input_data['demographic'] + ' ' + input_data['essay']
  
    return input_data

In [6]:
def save_preprocessed(df, dataname):
    raw_data = df.copy()
    processed_df = num_to_text(raw_data)
    processed_df.to_csv("./processed_data/preprocessed_" + dataname + ".csv")

In [7]:
# save_preprocessed(raw_train, dataname="train")
# save_preprocessed(train_dev, dataname="train_dev")
save_preprocessed(raw_dev, dataname="dev")
# save_preprocessed(complete_dev, dataname="complete_dev") #dev with output label
# save_preprocessed(raw_test, dataname="test")

Existing non-numeric columns:
 ['essay', 'split']


# Data augmentaiton (Paraphrasing) -- train and dev set

In [1]:
import pandas as pd
import os
import torch
import transformers as trf

In [2]:
os.chdir("/home/573/rh2942/WASSA-2023-EMP")

In [3]:
train_dev = pd.read_csv("./processed_data/preprocessed_train_dev.csv", index_col=0)

In [4]:
train_dev.columns

Index(['conversation_id', 'article_id', 'essay', 'speaker_id', 'gender',
       'education', 'race', 'age', 'income', 'speaker_number', 'split',
       'essay_id', 'empathy', 'distress', 'emotion',
       'personality_conscientiousness', 'personality_openess',
       'personality_extraversion', 'personality_agreeableness',
       'personality_stability', 'iri_perspective_taking',
       'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern',
       'article', 'article_id_text', 'demographic', 'demographic_essay'],
      dtype='object')

In [5]:
# keeping chosen columns only
train_dev = train_dev[['essay', 'empathy', 'distress', 'emotion',
       'personality_conscientiousness', 'personality_openess',
       'personality_extraversion', 'personality_agreeableness',
       'personality_stability', 'iri_perspective_taking',
       'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern', 'article', 'demographic', 'demographic_essay']]

In [6]:
train_dev.sample(2)

,essay,empathy,distress,emotion,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,article,demographic,demographic_essay
642,"After reading the article, i can't help but fe...",1.000000,1.625,Sadness,6.75,6.75,6.75,6.75,7.0,4.643,2.0715,4.143,4.643,"Celebrity jeweler's surrogate son, second man ...",I am male. My age is 29.0 years. My education ...,I am male. My age is 29.0 years. My education ...
768,I just read a story about an apparent murder s...,4.166667,1.250,Neutral,6.00,6.50,1.00,6.00,5.5,5.000,2.2860,3.286,4.143,"Wife Who Died Alongside Husband, Children in M...",I am male. My age is 29.0 years. My education ...,I am male. My age is 29.0 years. My education ...


In [95]:
paraphrased = train_dev.copy()

In [96]:
paraphrased.sample(2)

,essay,empathy,distress,emotion,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,article,demographic,demographic_essay
145,I'm mostly bothered by this story because of h...,1.5,3.50,Disgust/Sadness,2.5,2.0,1.5,2.0,1.0,3.286,4.429,2.714,2.571,"The Sungai Putri, meaning River of the Princes...",I am male. My age is 41.0 years. My education ...,I am male. My age is 41.0 years. My education ...
801,So I just read an article about how a father a...,5.0,4.75,Anger,6.0,6.0,5.5,6.5,3.0,4.857,3.143,2.571,4.857,A father and son have died after falling off a...,I am male. My age is 25.0 years. My education ...,I am male. My age is 25.0 years. My education ...


In [97]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = trf.AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = trf.AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

In [98]:
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=1,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7
):
    max_length=len(question) #length of existing sentence is the limit
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    outputs = model.generate(
        input_ids.to(device), temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [99]:
for index, row in paraphrased.iterrows():
    paraphrased.loc[index, "demographic"] = paraphrase(paraphrased.loc[index, "demographic"])[0] # 0-index to take the one and only first paraphrased item
    paraphrased.loc[index, "essay"] = paraphrase(paraphrased.loc[index, "essay"])[0]
    paraphrased.loc[index, "article"] = paraphrase(paraphrased.loc[index, "article"])[0]

/scratch/jr19/rh2942/miniconda3/lib/python3.10/site-packages/transformers/generation_beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


In [100]:
paraphrased["demographic_essay"] = paraphrased["demographic"] + paraphrased["essay"]

In [101]:
paraphrased

,essay,empathy,distress,emotion,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,article,demographic,demographic_essay
0,I am filled with sadness seeing people living ...,6.833333,6.625,Hope/Sadness,7.00,5.50,1.00,6.50,6.0,4.857,2.0000,3.429,5.000,The absence of food in the mountains is causin...,My profile details a black or African-American...,My profile details a black or African-American...
1,Why isn't there more effort to assist the peop...,5.833333,6.000,Anger,5.50,5.00,2.00,5.50,4.5,3.429,2.8570,2.857,2.714,The absence of food in the mountains is causin...,"I am a male with an age of 32.0 years, accompa...","I am a male with an age of 32.0 years, accompa..."
2,"After reading the article, I am filled with sa...",1.000000,1.375,Sadness,6.75,6.75,6.75,6.75,7.0,4.643,2.0715,4.143,4.643,The absence of food in the mountains is causin...,"I am a male with 290 years of age, four year b...","I am a male with 290 years of age, four year b..."
3,"Despite having an amazing story to tell, someo...",6.166667,6.625,Sadness,6.00,6.00,5.00,4.50,3.5,5.000,4.1430,4.857,5.000,"The Miami Marlins pitcher, Jose Fernández, who...","I am a female, two-year associate degree, whit...","I am a female, two-year associate degree, whit..."
4,It seems as though an all-around kind and gene...,6.833333,1.000,Neutral,7.00,5.50,1.00,6.50,6.0,4.857,2.0000,3.429,5.000,"The Miami Marlins pitcher, Jose Fernández, who...","I am a male with an age of 37.0 years, four ye...","I am a male with an age of 37.0 years, four ye..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,I'm feeling apprehensive about the individuals...,1.000000,1.875,Fear/Sadness,6.75,6.75,6.75,6.75,7.0,4.643,2.0715,4.143,4.643,The Portugal-bound train that derailed in Spai...,"My name is Male, I am 29.0 years old and I hav...","My name is Male, I am 29.0 years old and I hav..."
983,Despite my best efforts to buy food with purpo...,3.500000,3.625,Sadness,5.00,3.00,5.00,4.00,3.5,2.714,3.0000,3.143,3.286,The paper published in Plos One highlights the...,"As a female with XSW degrees and White skin, I...","As a female with XSW degrees and White skin, I..."
984,I wish the article had provided more informati...,4.000000,4.375,Neutral,5.00,3.00,5.00,4.00,3.5,2.714,3.0000,3.143,3.286,The police have arrived at the Azusa location ...,"My background is White, my age is 21.0 years, ...","My background is White, my age is 21.0 years, ..."
985,This is a truly shocking story. I find it asto...,6.000000,6.000,Fear,6.00,6.00,6.00,6.00,6.0,4.000,2.2860,3.571,3.714,A recent report has revealed that providers wh...,"I am a female, 27.0 years old, with an educati...","I am a female, 27.0 years old, with an educati..."


In [102]:
train_dev_paraphrased = pd.concat([train_dev, paraphrased], axis=0, ignore_index=True)

In [103]:
train_dev_paraphrased.to_csv("./processed_data/train_dev_paraphrased.csv")

## Separate augmented train to check dev performance separately

In [13]:
# train_dev_paraphrased = pd.read_csv("./processed_data/train_dev_paraphrased.csv", index_col=0, header=0)

In [104]:
train = train_dev_paraphrased.iloc[0:779,:]

In [108]:
train_paraphrased = train_dev_paraphrased.iloc[987:1766,:]

In [109]:
train_train_paraphrased = pd.concat([train, train_paraphrased], ignore_index=True, axis=0)

In [110]:
train_train_paraphrased.to_csv("./processed_data/train_train_paraphrased.csv")

# Check lengths

In [52]:
from datasets import Dataset

In [54]:
tokeniser = trf.AutoTokenizer.from_pretrained("bert-base-uncased")

#padding="longest" can be deferred to do dynamic padding
def tokenise(sentence):
    return tokeniser(sentence["demographic_essay"], truncation=True)

hugging_dataset = Dataset.from_pandas(train_dev, preserve_index=False)

tokenised_hugging_dataset = hugging_dataset.map(tokenise, batched=True)

Map:   0%|          | 0/987 [00:00<?, ? examples/s]

In [58]:
# checking length after tokenisation

length = []
for i in range(tokenised_hugging_dataset.num_rows):
  length.append(len(tokenised_hugging_dataset['input_ids'][i]))

print(f"Max length: {max(length)}")

Max lengths: 236


In [61]:
train_dev["demographic_essay"].str.len().max()

956

In [63]:
train_dev["article"].str.len().min()

284

In [10]:
train_dev["article"].str.len().max()

20047

In [8]:
train_dev["demographic_essay"].str.len().max()

956

In [14]:
train_dev_paraphrased["article"].str.len().max()

987

# Extra

## ChatGPT

In [ ]:
article_raw.iloc[0,1]

In [28]:
import openai

In [43]:
with open("./openai-api.txt", "r") as f:
    openai.api_key = f.read().strip("\n")

In [45]:
# model = "gpt-3.5-turbo"
model ="text-davinci-003"

In [ ]:
response = openai.Completion.create(
  model=model,
  prompt= article_raw.iloc[0,1] + "\n\nTl;dr",
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=1
)

## Parrot

In [ ]:
from parrot import Parrot

In [ ]:
paraphrased.loc[429,'essay']

'I am not surprised that the nations of Africa are having difficulty agreeing on conservation efforts for elephants. There is much disharmony politically between them. Africa is home to a number of endangered and threatened animals. I think the global community has to be involved in these conservation efforts.'

In [ ]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=True)

In [ ]:
parrot.augment(input_phrase=paraphrased.loc[429,'essay'], use_gpu=True, do_diverse=True, max_return_phrases=5)

[('I am not surprised that the nations of Africa are having difficulty agreeing on conservation efforts for elephants. There is much disharmony politically between them. Africa is home to a number of endangered and threatened animals. I think the global community has to be involved in these conservation efforts.',
  0)]

Current problem: the paraphrased sentence is pretty much the same as original

## Pegasus

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
def get_response(input_text,num_return_sequences,num_beams):
    batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [ ]:
get_response(input_text = paraphrased.loc[429,'essay'], num_return_sequences=1 ,num_beams=10)

["I am not surprised that the nations of Africa don't agree on elephants."]

Current problem: only the first sentence is paraphrased

## FastText embeddings

In [17]:
# essay_empathy.head()

In [18]:
import fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model('cc.en.300.bin')

In [19]:
essay_fasttext = pd.DataFrame(essay_empathy['essay'].apply(lambda x: ft.get_sentence_vector(x)))
# essay_fasttext

In [20]:
# expanding embedding list to columns
essay_fasttext = pd.concat([essay_fasttext.drop(columns='essay'), pd.DataFrame(essay_fasttext['essay'].tolist(), index=essay_fasttext.index).add_prefix('fasttext_')],axis=1)

In [21]:
# essay_fasttext

In [22]:
essay_empathy.drop(columns='essay', inplace=True) # no longer required

In [23]:
# essay_empathy

In [24]:
empathy_fasttext = pd.concat([essay_empathy, essay_fasttext], axis=1)

In [ ]:
empathy_fasttext

In [26]:
empathy_fasttext.to_csv('./empathy_fasttext.csv')